In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import time
import datetime

In [ ]:
import pandas as pd
epa = pd.read_csv('../datasets/epa_violations.csv')

In [ ]:
epa.head()

In [ ]:
cols = [col for col in epa.columns]
penalty_amt = 'Penalty Amount Adjusted For Eliminating Multiple Counting'

# create a new row with penalty amount as int
penalty_int = [int(entry.replace('$', '').replace(',','')) for entry in epa[penalty_amt]]
epa['penalty_amt'] = penalty_int

In [ ]:
cols_of_interest = ['penalty_amt', 'Company', 'Parent Company', 'Parent Company Stock Ticker', 'Penalty Year', 'Penalty Date']

In [ ]:
# create a new dataframe only with the above columns
epa_cols_interest = epa[cols_of_interest].copy()

In [ ]:
# drop NaN rows and reindex our table
epa_cols_interest = epa_cols_interest.dropna()
epa_cols_interest = epa_cols_interest.reset_index()

In [ ]:
# sort by highest fine
# epa_cols_interest.sort_values(by=['Parent Company Stock Ticker'], ascending=False)
epa_cols_interest.sort_values(by=['penalty_amt'], ascending=False)

In [ ]:
# Put violations into (ticker, date1, date2, amount)

def to_unix(t):
    t = str(t)
    add_slash = lambda x: x[:4]+'/'+x[4:6]+'/'+x[6:]
    t = add_slash(t)
    new_t = time.mktime(datetime.datetime.strptime(t, "%Y/%m/%d").timetuple())
    return new_t


def time_range(d,delta):
    yr_to_sec = lambda delta: delta*24*60*60
    prior = d - yr_to_sec(delta)
    post = d + yr_to_sec(delta)
    return (prior, post)

# are we using market cap here?
def date_range(df):
    for row in df.iterrows():
        data = row[1]
        date = data[6]
        ticker = data[2]
        penalty = data[1]
        unix_date = to_unix(date)    
        yield (penalty, ticker, time_range(unix_date, 2.5))

In [ ]:
test = date_range(epa_cols_interest)

In [ ]:
# TODO:
#   split fines by amount
# Fines here are split by billions, millions, thousands, and hundreds

fines_bill = epa_cols_interest[epa_cols_interest['penalty_amt'] > 999999999]
fines_mill = epa_cols_interest[epa_cols_interest['penalty_amt'].between(999999, 999999999)]
fines_thou = epa_cols_interest[epa_cols_interest['penalty_amt'].between(999, 999999)]
fines_hund = epa_cols_interest[epa_cols_interest['penalty_amt'].between(0, 998, inclusive=True)]

In [ ]:
def plot_years_sum(df):
    fine_by_year = []
    unique_years = df['Penalty Year'].unique()
    for year in unique_years:
        epa_years = df[df['Penalty Year'] == year]
        fine_by_year += [(sum(epa_years['penalty_amt']), year)]
        
    sorted_fines = sorted(fine_by_year, key = lambda x: x[1])
    x_axis = [year for _,year in sorted_fines]
    y_axis = [fine for fine,_ in sorted_fines]
    
    plt.plot(x_axis, y_axis)
    plt.title("Fines by Year")
    plt.show()

In [ ]:
plot_years_sum(fines_bill)
plot_years_sum(fines_mill)
plot_years_sum(fines_thou)
plot_years_sum(fines_hund)
plot_years_sum(epa_cols_interest)

In [ ]:
# TOP FINES BY YEAR (using main dataframe epa_cols_interest)
unique_years = sorted(epa_cols_interest['Penalty Year'].unique())
for year in unique_years:
    this_year = epa_cols_interest[epa_cols_interest['Penalty Year'] == year]
    penalty = list(this_year['penalty_amt'])
    ticker = list(this_year['Parent Company Stock Ticker'])
    max_ = max(zip(penalty, ticker))
    print(year, *max_)

<h1> not really running this </h1>

In [ ]:
sd = epa_cols_interest['penalty_amt'].std()
avg = epa_cols_interest['penalty_amt'].mean()

In [ ]:
#normalize values
z_score = lambda x: ((x - avg) / float(sd))

epa_cols_interest['normalized_amt'] = [z_score(val) for val in epa_cols_interest['penalty_amt']]

In [ ]:
test_set = sorted(epa_cols_interest['normalized_amt'])[::-1]
test_set

plt.plot(test_set)
plt.show()